# Load Data into a Database

* Install faker to fake PII data

In [1]:
%pip install faker

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
from faker import Faker # To fake PII data like names
import random

/usr/local/lib/python3.9/site-packages/pandas/compat/numpy/__init__.py:10: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _nlv = LooseVersion(_np_version)
/usr/local/lib/python3.9/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/usr/local/lib/python3.9/site-packages/pandas/compat/numpy/function.py:120: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(__version__) >= LooseVersion("1.17.0"):


In [3]:
fake = Faker()

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Connect to Database
* In reality the credentials would be stored in a secure way like with Secrets Mangager

In [4]:
DB_NAME = "legacy_pacheco"
DB_USER = "rpacheco"
DB_PASSWORD = "D1qoGvjXqPTd2vB4i4KS71xDEWmCOpSq"
DB_HOST = "dpg-d1ff63mmcj7s739p963g-a.oregon-postgres.render.com"
DB_PORT = "5432"

In [5]:
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT,
    sslmode="require"
)
cursor = conn.cursor()

## Read in Kaggle NLP Mental Health Conversations Dataset
* https://www.kaggle.com/datasets/thedevastator/nlp-mental-health-conversations?resource=download

In [6]:
df = pd.read_csv('train.csv')

In [7]:
df.head()

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...


* Assign fake PII data. I chose first/last name, clinic (city), doctor, and age
* This is to train a model to generate a numerical/categorical response

In [8]:
df["firstName"] = [fake.first_name() for _ in range(len(df))]
df["lastName"] = [fake.last_name() for _ in range(len(df))]
df["clinic"] = [fake.city() for _ in range(len(df))]
df["doctor"] = [fake.last_name() for _ in range(len(df))]
df["age"] = [int(random.randint(18, 50)) for _ in range(len(df))] # keeping ages in the 18 - 50 range so data isn't to spread out

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
df.head()

,Context,Response,firstName,lastName,clinic,doctor,age
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb...",Dillon,Clark,North Wendy,Thomas,28
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see...",Shelby,Lamb,North Heather,Bell,45
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...,Blake,Estrada,East Jasmineton,Brewer,44
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...,Amber,Simpson,New Justinchester,Stewart,41
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...,Kenneth,Lee,Isaacchester,Fuller,49


* Convert dataframe to a numpy array to insert into postgres

In [10]:
data = df.to_records(index=False).tolist()

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


* Generate insert query and insert into table

In [11]:
insert_query = """
    INSERT INTO records (context, response, firstName, lastName, clinic, doctor, age)
    VALUES %s
"""
execute_values(cursor, insert_query, data)

conn.commit()

In [12]:
cursor.close()
conn.close()